# Preguntas - Desafío ML Engineer 

## Preliminares

> Antes de responder a las preguntas planteadas en el desafío, mencionaré algunas modificaciones realizadas al trabajo entregado por Juan, ya que originalmente no compilaba correctamente.
>
> 1. Error en la generación de gráficos de barra: El error entregado es el siguiente:` TypeError: barplot() takes from 0 to 1 positional arguments but 2 were given`. Solución: Poner explícitamente que el primer valor entregado al método corresponde a x, mientras que el segundo corresponde a y `f(x=.., y=..)`.
>
>2. Error al llamar al método `calcular_tasa`: Específicamente, el error se produce en la siguiente linea de código: `tasas_destinos = calcular_tasa(df, 'SIGLADES')`, entregando el siguiente mensaje: `'Series' object has no attribute 'iteritems'`. Solución: Dentro de la función, el error ocurre al querer iterar la Serie de Frecuencias creada utilizando iteritems(). Esto puede deberse a un problema en la versión de pandas, simplemente reemplazar la forma en que se itera por la siguiente: `for name, total in total_values.items():`.
>
>3. Error al definir las etiquetas para el modelo XGBoost: El problema se da en la siguiente linea de código `labels_ = labels`. Solución: Reemplazar por
`labels_ = label`.
>
> Corregidos estos errores, a continuación se responde a las preguntas.

## Pregunta 1

Escoger el modelo que a tu criterio tenga un mejor performance, argumentando la decisión.

> **Respuesta**: El modelo escogido teniendo en cuenta sólo la información proporcionada en el notebook entregado por Juan, es el modelo XGBoostClassifier. La razón de escoger este modelo es simplemente la cantidad de experimentos realizados con este enfoque por sobre la regresión logística. En particular, se validó el rendimiento del modelo utilizando una búsqueda de hiperparámetros con GridSearch, en conjunto con la técnica de Cross-Validation. La decisión no se basó en las métricas debido a que no están bien planteadas, así que se priorizó más la potencia conocida del modelo por sobre los resultados de los experimentos. 
>
>A continuación explico por qué a mi parecer los experimentos no permiten tomar una decisión en base a las métricas entregadas:
>
> - Como se mencionó, existe un claro desbalance entre la clase positiva (delay) y la clase negativa. Esto implica que utilizar la métrica accuracy tradicional, beneficiaría a la clase mayoritaria, ocultando el rendimiento del modelo en la clase minoritaria. En particular, sería interesante analizar métricas que tomen en cuenta el desbalance como *balanced accuracy, precision, recall y f1-score*.
>
> - En segundo lugar, me parece correcto explorar técnicas de re-sampling para poder manejar el desbalance de clases. En este caso, se probó utilizando un OverSampling sobre la clase minoritaria. En primer lugar, creo que es importante explorar otras técnicas similares como UnderSampling, SMOTE o una combinación de las anteriores. Por otro lado, hay un error conceptual en la implementación; el oversampling se aplicó sobre todo el DataFrame, distorsionando así la distribución de etiquetas en la partición de test. Se sugiere aplicar estos métodos sólo al conjunto de entrenamiento.
>
> - Un último detalle a corregir, es que la ejecución del GridSearch en conjunto con el CrossValidation se realizó con `cv=2`. Esto no es una práctica común ya que en ese caso se generarían dos subconjuntos de test disjunto con el 50% de los datos, una porción distinta a los experimentos realizados anteriormente con el tercio de datos. Mínimo se sugiere aplicar `cv=3`.
>
>Dicho esto, en la siguiente pregunta se exploran posibles mejoras a la solución actual.

## Pregunta 2

Implementar mejoras sobre el modelo escogiendo la o las técnicas que prefieras.

> **Respuesta**: En el notebook `to-expose-v2.0.ipynb`, pueden encontrar una nueva versión del desafío, que corresponde a una mejora al notebook entregado por Juan incorporando modificaciones como las explicadas en la Pregunta 1. Gracias a los resultados de estos nuevos experimentos, se determinó que el modelo con mejores resultados fue de todas maneras el *XGBClassifier*, obteniendo un f1-score (micro o macro ya que es clasificación binaria) de 0.41 y un balance accuracy de 0.65. Por otro lado, los atributos con mayor importancia fueron el mes, los vuelos paralelos y la hora, siendo estos últimos incorporados en la nueva versión.
>
>En resumen, se consideran las siguientes modificaciones:
>
> 1. Métricas: Considerando el desbalance de clases, se consideron las siguientes métricas para comparar el rendimiento de los modelos respecto a la clasificación binaria; balanced accuracy, precision, recall y f1-score.
> 2. Balance de clases: Se probó utilizando las técnicas de undersampling, oversampling, SMOTE y la combinación de SMOTE con undersampling. Todo esto se realizó sólo sobre el conjunto de entrenamiento, sin alterar la distribución en el conjunto de test.
> 3. Manejo de distintos tipos de datos: Se probaron diferentes formas de codificar los atributos categóricos; LabelEncoder y OneHotEncoder, y atributos numéricos; StandardScaler y MinMaxScaler.
> 4. Selección de Modelos: Se probaron los siguiente modelos para resolver la clasificación binaria; LogisticRegression, RandomForestClassifier, GaussianNB, SGDClassifier, KNeighborsClassifier, BernoulliNB y XGBClassifier.
> 5. Optimización de Hiperparámetros: Para un subconjunto de los modelos anteriores (por costo computacional), se realizó un GridSearch con un Cross Validación (`cv=5`, para 20% de test en cada partición disjunta).



## Pregunta 3
Exponer el modelo seleccionado como API REST para ser expuesto.

> **Respuesta:** El mejor modelo (XGBClassifier), fue almanecado en un archivo pickle al final del notebook  `to-expose-v2.0.ipynb`. Con más tiempo, la idea sería crear un Script en el repositorio que tenga la lógica del entrenamiento y almacenado del mejor modelo, por ahora queda en el formato de notebook.
>
> Algunos detalles importantes:
>
> 1. En la carpeta models/ se almacenará tanto el modelo pre-entrenado como el archivo que contiene la lista de columnas generadas al realizar las transformaciones de datos en el entrenamiento. Esto es importante ya que al realizar predicciones de a un elemento, no siempre se generarán la misma cantidad de columnas con el método `get_dummies`, por lo que hay que ser consistentes. Otra opción aquí era crear un Pipeline donde las variables categóricas fueran transformadas con la Clase OneHotEncoder, sin embargo, en la documentación aparece que es más lento este último método. 
> 2. Para poder ejecutar la API que consumirá este modelo, es necesario instalar todas las dependencias del archivo `requirements.txt` con el comando `pip install -r requirements.txt`. Para evitar realizar esto constantemente, se dejó el archivo Docker necesario para crear una imagen con este ambiente.  
> 3. El Script que contiene toda la lógica de la API es  `app.py`. Para este desafío se eligió utilizar la librería FastAPI, frecuentemente utilizada para crear APIs de modelos de Machine Learning con una sintaxis muy simple. En el archivo se encuentran los pasos debidamente documentados.
> 4. Ejecutar el comando `python app.py`. Para hacer peticiones hay que utilizar la siguiente URL:[http://localhost:8000/](http://localhost:8000/). 
> 5. Existen tres formas simples de realizar una petición y obtener la predicción del modelo. 1. Ejecutar el comando `python main.py` ya que en `main.py` se dejó una petición lista para ser enviada. 2. Utilizar interfaz interactiva que nos entrega FastAPI en la siguiente URL: `http://localhost:8000/docs`. 3. Con el comando curl como en el ejemplo que se deja al final de la respuesta.
>
```
curl -X 'POST'
'http://localhost:8000/predict'
-H 'accept: application/json'
-H 'Content-Type: application/json'
-d '{ "MES": 7, "temporada_alta": 1, "hora": 2, "vuelos_paralelos": 20, "OPERA": "American Airlines", "SIGLADES": "Miami", "DIANOM": "Lunes", "periodo_dia": "tarde" }'
```



## Pregunta 4
Hacer pruebas de estrés a la API con el modelo expuesto con al menos 50.000 requests durante 45 segundos. Para esto debes utilizar esta herramienta y presentar las métricas obtenidas.

> **Respuesta:** Simplemente se siguen las instrucciones de instalación de la librería wrk, en mi caso para el SO MAC OS: `brew install wrk --build-from-source --HEAD`
> 
> Se ejecuta el siguiente comando: `wrk -t 20 -c 2500 -d 45s -s post.lua http://localhost:8000/predict` el cuál permite enviar 2500 request en cada uno de los 20 hilos solicitados, complementando las 50.000 pedidas en la pregunta.
>
> En cuanto a la eficiencia de la API, se pudo manejar un promedio de 7.25 solicitudes por segundo con un máximo de 141 solicitudes por segundo. El tiempo de latencia promedio fue de 1.16 segundos, con un tiempo máximo de 2 segundos y una desviación estándar de 419.48ms. Además, se leyeron 472.83 KB y se realizaron 3433 solicitudes en total. Teniendo en cuenta estos resultados, la API puede considerarse funcional pero su rendimiento puede mejorarse para manejar una carga de trabajo mayor. No es factible utilizarla en producción, debe ser optimizada.



## Pregunta 5

¿Cómo podrías mejorar el performance de las pruebas anteriores?

> **Respuesta:** Limitado a mi conocimiento actual, creo que las siguientes podrían ser posibles soluciones para mejorar el rendimiento de la API actual:
>
> Cargar el modelo una sóla vez: Actualmente, tanto el modelo como los nombres de las columnas se cargan cada vez que se realiza una predicción. La idea sería cargar una sóla vez estos archivos y guardarlos en memoria.
>
> Utilizar código asíncrono: Por defecto, FastAPI utiliza un servidor web asíncrono (es decir, uvicorn) y admite código asíncrono. En este punto el código implementado lo que hace es bloquear el resto del código hasta que ciertas tareas son completadas (por ejemplo la carga de los modelos). Una posible mejora sería utilizar la sintaxis async/await y librerías asíncronas.
>
> Utilizar un caché: Se podría utilizar un caché para almacenar respuestas ya computadas, aunque esto podría aumentar la complejidad de la aplicación y no ser una buena práctica si es modelo se va re-entrenando.
>
> Utilización de la nube: Se podría considerar usar una solución de alojamiento en la nube en lugar de ejecutar el servidor de forma local en el propio computador. Los proveedores de la nube, como AWS o Google Cloud, ofrecen servicios de alojamiento de aplicaciones web que están optimizados para manejar grandes cargas de tráfico y solicitudes simultáneas, lo que permitiría optimizar los tiempos.
>
> Implementación de método predict desde 0: Los métodos predict creados para la mayoría de los modelos presentes en Scikit-Learn no sólo ejecutan la predicción sobre los nuevos datos, sino que previamente realizan una verificación de los datos de entrada (si es un numpy 2D array, si coinciden los atributos con los del modelo entregado, si no hay nulos). Esto aumenta el tiempo de computo de las predicciones y la latencia del sistema. La solución a esto sería crear una clase que extienda a la original, y redefinir el método `predict`. Por ejemplo, para el caso de la Regresión Logística, bastaría con computar la multiplicación entre el valor de entrada y el coeficiente encontrado, sumándole el valor de la intercepción. Luego, se aplica la función softmax (ojalá implementada desde cero también, por las mismas razones).
>
> Cambiar el modelo por uno con resultados similares pero menor complejidad.
